## Imports

In [1]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm
from sklearn.utils.class_weight import compute_class_weight

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render
from ax.utils.tutorials.cnn_utils import train, evaluate

from imblearn.over_sampling import SMOTE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Loading Data

In [2]:
data_train_path = 'skin-lesions/train/'
data_valid_path = 'skin-lesions/valid/'
data_test_path = 'skin-lesions/test/'

https://discuss.pytorch.org/t/is-there-a-limit-on-how-disbalanced-a-train-set-can-be/26334/6?u=ptrblck

Below is an example of the undersampler that we did not end up using.

In [4]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

# train_dataset = torch.load('model_data/train_dataset.pt')
train_dataset = torch.load('upd_train_data.pt')

## UNDERSAMPLER 

# labels = []
# for idx, (sample, target) in enumerate(tqdm(train_dataset, total=len(train_dataset))):
#     labels.append(int(target))
# cls_weights = torch.from_numpy(
#     compute_class_weight('balanced', classes=np.unique(labels), y=labels)
# )

# weights = cls_weights[labels]
# sampler = WeightedRandomSampler(weights, len(labels), replacement=True)

# print(cls_weights)
# print(np.unique(labels))

# # train_dataset = ImageFolder(data_train_path, transform = train_transform)

valid_dataset = ImageFolder(data_valid_path, transform = test_transform)
test_dataset = ImageFolder(data_test_path, transform = test_transform)

# train_loader = DataLoader(train_dataset, sampler = sampler, batch_size = 64)
valid_loader = DataLoader(valid_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle = True)

## SMOTE

In [5]:
modified_outputs, labels = [], []
for idx, (sample, target) in enumerate(tqdm(train_dataset, total=len(train_dataset))):
    modified_outputs.append(sample.cpu().detach().numpy())
    labels.append(target)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [6]:
X_train, y_train = np.array(modified_outputs), np.array(labels)

https://stackoverflow.com/questions/53666759/use-smote-to-oversample-image-data

In [7]:
sm = SMOTE(random_state=42)

train_rows=len(X_train)
X_train = X_train.reshape(train_rows,-1)

X_train, y_train = sm.fit_resample(X_train, y_train)
X_train = X_train.reshape(-1, 3, 224, 224)

In [8]:
print(np.unique(y_train, return_counts=True))

(array([0, 1]), array([1626, 1626]))


In [9]:
train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)

## CNN

In [10]:
class ConvolutionalNetwork(torch.nn.Module):
    def __init__(self):
        super(ConvolutionalNetwork, self).__init__()
        self.model = nn.Sequential(
            
            #input: (3, 224, 244), output: (6, 190, 190)
            torch.nn.Conv2d(3, 6, 35),
            torch.nn.ReLU(),
            #input: (6, 190, 190), output: (6, 95, 95)
            torch.nn.MaxPool2d(2, 2),
            
            torch.nn.Dropout(0.4),

            #input: (6, 95, 95), output: (16, 61, 61)
            torch.nn.Conv2d(6, 16, 35),
            torch.nn.ReLU(),
            #input: (16, 61, 61), output: (16, 30, 30)
            torch.nn.MaxPool2d(2, 2),
            
            torch.nn.Dropout(0.4),

            #input: (16, 30, 30), output: (32, 11, 11)
            torch.nn.Conv2d(16, 32, 20),
            torch.nn.ReLU(),
            #input: (32, 11, 11), output: (32, 5, 5)
            torch.nn.MaxPool2d(2, 2),
            
            torch.nn.Dropout(0.2),

            torch.nn.Flatten(),
            torch.nn.Linear(32 * 5 * 5, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 84),
            torch.nn.ReLU(),
            torch.nn.Linear(84, 2),
        )

    def forward(self, x):
        return self.model(x)

## Utility Functions

In [11]:
def train(model, epochs, criterion, min_loss, optimizer, vectorize=False):
    training_losses, valid_losses, accs = [],[],[]
    for epoch in range(epochs):
        training_loss = 0
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            if vectorize:
                images = images.reshape(-1, 224 * 224 * 3)

            optimizer.zero_grad()
            ps = model(images)
            loss = criterion(ps, labels)
            loss.backward()
            optimizer.step()

            training_loss += loss.item()
        print(f"\tEPOCH: {epoch + 1}.. TRAINING LOSS: {training_loss}")

        training_losses.append(training_loss)
        model.eval()
        valid_loss = 0
        acc = 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                
                if vectorize:
                    images = images.reshape(-1, 224 * 224 * 3)
                
                optimizer.zero_grad()
                ps = model(images)
                loss = criterion(ps, labels)
                
                valid_loss += loss.item()
                
                _, top_class = ps.topk(1, dim = 1)
                eq = top_class == labels.view(-1, 1)
                acc += eq.sum().item()
                
        valid_losses.append(valid_loss)
        accs.append(acc)
        acc = (acc/len(valid_dataset)) * 100
        print("EPOCHS: {}/{}.. \tTRAINING LOSS: {:.6f}.. \tVALIDATION LOSS: {:.6f}.. \tACCURACY: {:.2f}%..".format(epoch + 1, epochs, training_loss, valid_loss, acc))
        
        if valid_loss <= min_loss:
            print("Saving Model {:.4f} ---> {:.4f}".format(min_loss, valid_loss))
            save_obj = OrderedDict([
                ("min_loss", valid_loss),
                ("model", model.state_dict())
            ])
            torch.save(save_obj, "/melanoma_model.pt")
            min_loss = valid_loss
            
    return training_losses, valid_losses, accs

In [ ]:
def test_model():
    classes = test_dataset.classes
    total_correct = 0
    count = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            ps = model(images)
            
            ps = nn.Softmax(dim = 1)(ps)
            
            top_p, top_class = ps.topk(2, dim = 1)
            eq = top_class == labels.view(-1, 1)
            total_correct += eq.sum().item()
            
            if count % 50 == 0:
                plt.imshow(transforms.ToPILImage()(images[0]))
                plt.title(f"P: {classes[top_class.item()]}.. C: {top_p.item() * 100}%.. GT: {classes[labels.item()]}..")
                plt.show()
            
            
                
            count += 1
                
    print("Total Correct: {}/{}".format(total_correct, len(test_dataset)))
    print("Total Accuracy: {:.2f}%".format((total_correct/len(test_dataset)) * 100))

In [12]:
model = ConvolutionalNetwork().to(device)
model

ConvolutionalNetwork(
  (model): Sequential(
    (0): Conv2d(3, 6, kernel_size=(35, 35), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.4, inplace=False)
    (4): Conv2d(6, 16, kernel_size=(35, 35), stride=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.4, inplace=False)
    (8): Conv2d(16, 32, kernel_size=(20, 20), stride=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout(p=0.2, inplace=False)
    (12): Flatten(start_dim=1, end_dim=-1)
    (13): Linear(in_features=800, out_features=512, bias=True)
    (14): ReLU()
    (15): Linear(in_features=512, out_features=128, bias=True)
    (16): ReLU()
    (17): Linear(in_features=128, out_features=84, bias=True)
    (18): ReLU()
    (19): Linear(in_features=84, out_features=2, bias=True)
  )
)

## Bayesian Optimizer

In [35]:
def init_net(parameterization):

    model = ConvolutionalNetwork().to(device)

    for param in model.parameters():
        param.requires_grad = False # Freeze feature extractor
        
    return model # return untrained model


def net_train(net, train_loader, parameters, dtype, device):
    net.to(dtype=dtype, device=device)

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), # or any optimizer you prefer 
                        lr=parameters.get("lr", 0.001), # 0.001 is used if no lr is specified
                        weight_decay=parameters.get('weight_decay', 0.1)
    )

    scheduler = optim.lr_scheduler.StepLR(
      optimizer,
      step_size=int(parameters.get("step_size", 30)),
      gamma=parameters.get("gamma", 1.0),  # default is no learning rate decay
    )

    num_epochs = parameters.get("num_epochs", 2) # Play around with epoch number
    # Train Network
    for _ in range(num_epochs):
        for inputs, labels in train_loader:
            # move data to proper dtype and device
            inputs = inputs.to(dtype=dtype, device=device)
            labels = labels.to(device=device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.requires_grad=True
            loss.backward()
            optimizer.step()
            scheduler.step()
        return net


def train_evaluate(parameterization):

    # constructing a new training data loader allows us to tune the batch size
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                batch_size=parameterization.get("batchsize", 32),
                                shuffle=True)
        
    # Get neural net
    untrained_net = init_net(parameterization)
    
    # train
    trained_net = net_train(net=untrained_net, train_loader=train_loader, 
                            parameters=parameterization, dtype=torch.float, device=device)
    
    # return the accuracy of the model as it was trained in this run
    return evaluate(
        net=trained_net,
        data_loader=test_loader,
        dtype=torch.float,
        device=device,
    )


In [ ]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True},
        {"name": "batchsize", "type": "range", "bounds": [16, 128]},
        {"name": "weight_decay", "type": "range", "bounds": [0.0, 0.1]},
#         {"name": "max_epoch", "type": "range", "bounds": [1, 30]},
#         {"name": "stepsize", "type": "range", "bounds": [20, 40]},        
    ],
  
    evaluation_function=train_evaluate,
    objective_name='accuracy',
)

print(best_parameters)
means, covariances = values
print(means)
print(covariances)

## Training Loop

Below is not the full training loop. Full training loop was run on Google Colab and the instance was lost. 

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 68, shuffle = True)

In [ ]:
torch.cuda.empty_cache()

model = ConvolutionalNetwork().to(device)

LEARNING_RATE = 3.470303960695998e-05
EPOCHS = 50

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

min_loss = -1
train_loss, valid_loss, accs = train(model, EPOCHS, criterion, min_loss, optimizer)

	EPOCH: 1.. TRAINING LOSS: 35.20852941274643
EPOCHS: 1/50.. 	TRAINING LOSS: 35.208529.. 	VALIDATION LOSS: 3.287747.. 	ACCURACY: 66.67%..
	EPOCH: 2.. TRAINING LOSS: 34.93632787466049
EPOCHS: 2/50.. 	TRAINING LOSS: 34.936328.. 	VALIDATION LOSS: 3.409671.. 	ACCURACY: 54.00%..
	EPOCH: 3.. TRAINING LOSS: 34.83892202377319
EPOCHS: 3/50.. 	TRAINING LOSS: 34.838922.. 	VALIDATION LOSS: 3.390417.. 	ACCURACY: 54.67%..
	EPOCH: 4.. TRAINING LOSS: 34.7320693731308
EPOCHS: 4/50.. 	TRAINING LOSS: 34.732069.. 	VALIDATION LOSS: 3.459699.. 	ACCURACY: 48.00%..
	EPOCH: 5.. TRAINING LOSS: 34.5806400179863
EPOCHS: 5/50.. 	TRAINING LOSS: 34.580640.. 	VALIDATION LOSS: 3.363219.. 	ACCURACY: 54.67%..
	EPOCH: 6.. TRAINING LOSS: 34.5492085814476
EPOCHS: 6/50.. 	TRAINING LOSS: 34.549209.. 	VALIDATION LOSS: 3.515167.. 	ACCURACY: 46.00%..
	EPOCH: 7.. TRAINING LOSS: 34.43019449710846
